# Leap Frog Time Stepping for Yang Mills

See work of Christiansen, Winther
https://epubs.siam.org/doi/pdf/10.1137/040616887

Srinath Bulusu, Joachim Schöberl

In [ ]:
from ngsolve import *

from ngsolve.meshes import *

from ngsolve.solvers import *
from ngsolve.webgui import Draw

import matplotlib.pyplot as plt

from su2_yangmills import *

In [ ]:
n = [1,0,0]
t = 10
v = CF( (t, CF(tuple(n))) )
Cross(v[1:], v[1:]).dim

In [ ]:
dim = 2
if dim == 2:
    mesh = MakeStructured2DMesh(quads=False, nx=30, ny=30, periodic_x=True, periodic_y=True)
elif dim == 3:
    mesh = MakeStructured3DMesh(hexes=False, nx=5, ny=5, nz=5, periodic_x=True, periodic_y=True, periodic_z=True)

#Draw(mesh)

Function spaces of Interest are Lie algebra valued p-forms $\Lambda^p(M,\mathfrak{g})$. In two dimensions $p=0,1,2$

The Lie algebra $\mathfrak{g} = su(2)$ can be spanned by three basis vectors (Pauli matrices $\sigma_i$ for example). We will from now on only work with coefficients in this basis, hence $\mathbb{R}^3$-valued p-forms $\Lambda^p(M,\mathbb{R}^3)$. 

In [ ]:
#fesC = H1(mesh)**3 # for su2 charge
#fesA = HCurl(mesh)**3 # for su2 gauge potential

fesC = Periodic(H1(mesh, order=1))**3 # for su2 charge
fesA = Periodic(HCurl(mesh, order=1))**3 # for su2 gauge potential

#fesC = Periodic(H1(mesh, order=3))**3 # for su2 charge
#fesA = Periodic(H1(mesh, order=3)**3)**3 # for su2 gauge potential


#print(fesA.FreeDofs())
#fesA.ndof

The connection is a map $\mathrm{d}_A: \Lambda^p(M,\mathfrak{g}) \rightarrow \Lambda^{p+1}(M,\mathfrak{g})$ and acts as $v \mapsto \mathrm{d}_A v := \mathrm{d}v + [A \wedge v]$

The exterior derivative in $\mathbb{R}^2$ can be interpreted as gradient and curl
$H1 \simeq \Lambda^0 \stackrel{\nabla}{\rightarrow} Hc \simeq \Lambda^1 \stackrel{\nabla \times}{\rightarrow} L2 \simeq \Lambda^2$

The wedge Bracket $[A \wedge B]$ is the linear extension of $[\mathrm{d}x^\mu \otimes \sigma_k \wedge \mathrm{d}x^\nu \otimes \sigma_l] = \mathrm{d}x^\mu \wedge \mathrm{d}x^\nu \otimes [\sigma_k, \sigma_l]$

The Lie Bracket cna be expressed in terms of structure constants $[\sigma_k, \sigma_l] = f_{klm} \sigma_m$ which in the case of $su(2)$ are the fully antisymmetric Levi Civita symbol $f_{klm}=\epsilon_{klm}$, and we can use the $\mathbb{R}^3$-Cross product for coefficients.

the inner product of two basis matrices is $\langle \sigma_k, \sigma_l \rangle = \mathrm{Tr}(\sigma_k \sigma_l^\top)$ which in the case of $su(2)$ is $\langle \sigma_k, \sigma_l \rangle = \delta_{kl}$, and we can use the $\mathbb{R}^3$-Inner Product for coefficients.

In [ ]:
E = fesA.TrialFunction()
A = fesA.TrialFunction()
Ap = fesA.TestFunction()
Ep = fesA.TestFunction()

C = fesC.TrialFunction()
Cp = fesC.TestFunction()

gfA = GridFunction(fesA)
gfE = GridFunction(fesA)
gfC = GridFunction(fesC)

#J_gf = GridFunction(fesA)
#J_gf.Set((0,0 ,0,0 ,0,0))
#Jz_gf = GridFunction(fesAz)
#Jz_gf.Set((0 ,0 ,0))

grad_A(A, Cp)
curl_A(A, E)

The $d+1$ strong formulation of the time dependent Yang Mills equations with charge density $\rho$ and spatial current $j$ in temporal gauge

$ \mathrm{d}_A^* \dot{A} = \rho$

$\ddot{A} + \mathrm{d}_A^* F(A) = j$

in weak form with $E = -\dot{A}$ are

$ \langle E ,d_A \rho^\prime \rangle = \langle \rho, \rho^\prime \rangle$

$\langle \dot{E}, A^\prime \rangle - \langle F(A), \mathrm{d}_A A^\prime \rangle = \langle j, A^\prime \rangle$

for all test functions $\rho^\prime \in \Lambda^0(M,\mathfrak{g}), A^\prime \in \Lambda^1(M,\mathfrak{g})$

The second equation is used for time steps.

The first equation is used to evaluate the charg at each step.


In [ ]:
# matrices for E-A leapfrog time stepping
m = BilinearForm(fesA)
m += InnerProduct(E,Ep)*dx

m.Assemble()
minv = m.mat.Inverse(fesA.FreeDofs())

b = BilinearForm(fesA, nonassemble=True)
b += InnerProduct(F_A(A), curl_A(A, Ep))*dx

f = LinearForm(fesA)
#f += InnerProduct(J_gf, Ep)*dx
f.Assemble()

In [ ]:
# matrices for evaluating charge
mC = BilinearForm(fesC)
mC += InnerProduct(C,Cp)*dx

mC.Assemble()
mCinv = mC.mat.Inverse(fesC.FreeDofs())

bC = BilinearForm(trialspace=fesA, testspace=fesC, nonassemble=True)
bC += InnerProduct(E, grad_A(gfA, Cp))*dx

In [ ]:
# TODO: set initial conditions that satisfy physical constraints
sigma2 = 0.02
mu1 = [0.7, 0.5]
mu2 = [0.3, 0.5]
mu3 = [0.5, 0.3]

if dim == 2:
    gfA.Set((0,0 ,0,0 ,0,0))
    #gfE.Set((0,0 ,0,0 ,0,0))
    #gfE.Set((0,gaussx(mu1[0], sigma2) ,0,gaussx(mu2[0], sigma2) ,0,0))
    #gfE.Set((0,gaussxy(mu1, sigma2) ,0,gaussxy(mu2, sigma2) ,gaussxy(mu3, sigma2),0))
    gfE.Set((0,gaussxyper(mu1, sigma2) ,0,gaussxyper(mu2, sigma2) ,gaussxyper(mu3, sigma2),0))
    
if dim == 3:
    gfA.Set((0,0,0 ,0,0,0 ,0,0,0))
    #gfE.Set((0,0,gaussxy(mu1, sigma2) ,0,0,gaussxy(mu2, sigma2) ,0,0,0))
    gfE.Set((0,0,gaussxyper(mu1, sigma2) ,0,0,gaussxyper(mu2, sigma2) ,0,0,0))

    
    

gfC0 = GridFunction(fesC)
tempC0 = gfC0.vec.CreateVector()
tempC0[:] = 0.

bC.Apply(gfE.vec, tempC0)
gfC0.vec.data = mCinv * tempC0

Draw (gfC0[:], mesh )

Draw (gfA[0,:], mesh )
Draw (gfA[1,:], mesh )
Draw (gfA[2,:], mesh )

Draw (gfE[0,:], mesh )
Draw (gfE[1,:], mesh )
Draw (gfE[2,:], mesh )





#TODO: given charge and spatial gauge - find E?
a = BilinearForm(testspace=fesC, trialspace=fesA, nonassemble=True)
a += InnerProduct(E, grad_A(gfA, Cp))*dx

fa = LinearForm(fesA)
fa += rho_cf * Cp *dx

#a.Assemble()
#fa.Assemble()

fa = LinearForm(fesA)
fa += rho_cf * Cp *dx



E_gf = GridFunction(fesA)
#TODO: solve generalized poisson equation
#E_gf.vec = a.mat.Inv() * fa.vec

In [ ]:
tend = 4.
dt = 0.001
t = 0.
n_steps = int(tend/dt)
n_steps

In [ ]:
scenes = []
scenes.append(Draw (gfC, mesh ))
for i in (0,1,2):
    scenes.append(Draw (gfA[i,:], mesh ))
    scenes.append(Draw (gfE[i,:], mesh ))

temp = gfA.vec.CreateVector()
temp[:] = 0.
tempC = gfC.vec.CreateVector()
tempC[:] = 0.

dq =[]

# visualize with trace-visualizer vite
# https://docu.ngsolve.org/latest/i-tutorials/unit-1.9-taskmanager/taskmanager.html
with TaskManager(): # pajetrace=10**8):
    for n in range(n_steps):
        #timestep
        gfA.vec.data -= dt * gfE.vec
        
        b.Apply(gfA.vec, temp)
        temp += f.vec
        gfE.vec.data += dt * minv * temp


        if (n >= 0):
            #solve for charge
            #bC.Assemble()
            #bC = BilinearForm(testspace=fesC, trialspace=fesA, nonassemble=True)
            #bC += InnerProduct(E, grad_A(gfA, Cp))*dx
            bC.Apply(gfE.vec, tempC)
            gfC.vec.data = mCinv * tempC
            dq.append([Integrate(Norm(gfC - gfC0), mesh) for c in range(liedim)])

            #TODO: improved schemes for charge at n-1 th step


        t += dt
        #input()
        if n % 10 == 0:
            for scene in scenes:
                scene.Redraw()
            print ("\r", t, end="")



In [ ]:
plt.plot(range(0,len(dq)),dq)
plt.savefig("per2d.png")